### 56GB 데이터세트 사이즈 줄이기
1. 데이터세트 폴더에 있는 모든 디렉토리 리스트 조회
2. 이미지가 있는 폴더만 이미지 사이즈 줄여서 따로 저장 

In [53]:
from torch.utils.data import Dataset
from PIL import Image
import os
import numpy as np

In [22]:
# 데이터세트 폴더
SOURCE_PATH = './data/train'
SOURCE_PATH_LIST = os.listdir(SOURCE_PATH)

# 따로 저장할 폴더
TARGET_PATH = './trans/'

SOURCE_PATH_LIST

['cats', 'dogs', 'temp']

In [35]:
'cat.0.jpg'.endswith(('.jpg', '.tiff','.tif'))

True

In [68]:
os.path.join(SOURCE_PATH,SOURCE_PATH_LIST[0],'cat.jpg')

'./data/train\\cats\\cat.jpg'

In [58]:
image_path = []
file = []
dic = dict()
for source in SOURCE_PATH_LIST:
    dir = os.path.join(SOURCE_PATH,source)
    files = os.listdir(dir)
    # 파일이 있는 이미지 디렉토리 이름만 추출
    if 0 < len(files):
        if files[0].endswith(('.jpg','.tif','.tiff')):
            image_path.append(source)
            file.append(files)
            dic.update({source:files})
image_path  
a = np.reshape(list(dic.values()),(-1,))
a.shape

(600,)

In [67]:
for key in dic.keys():
    print(key)

cats
dogs


In [89]:
class SizeTransDataset(Dataset):
    def __init__(self, source_path, shape=(256,256), max_files=None) :
        self.source_path = source_path
        self.source_path_list = os.listdir(source_path)
        self.shape = shape
        self.max_files = max_files

        self.dic = dict()
        
        for source in self.source_path_list:
            files = os.listdir(os.path.join(self.source_path,source))
            if 0 < len(files):
                if files[0].endswith(('.jpg','.tif','.tiff')):
                    self.dic.update({source:files})

        self.total_files = np.reshape(list(self.dic.values()),(-1,))

    def __getitem__(self, index):
        return self.total_files[index]

    def __len__(self):
        return len(self.total_files)

    def trans(self,target_path):
        for key in self.dic.keys():
            target_dir = os.path.join(target_path,key)
            os.makedirs(target_dir,exist_ok=True)
            c = 1
            for f in self.dic[key]:
                file_path = os.path.join(self.source_path,key,f)
                with Image.open(file_path) as file:
                    image = file.resize(self.shape)
                    image.save(os.path.join(target_dir,f))
                if self.max_files is not None:
                    if self.max_files <= c:
                        break
                c+=1
            print(f"{key} : {c} files resized.")


In [90]:
testset = SizeTransDataset(SOURCE_PATH,max_files=5)
testset.trans(TARGET_PATH)

cat.0.jpg
cat.1.jpg
cat.10.jpg
cat.100.jpg
cat.101.jpg
cats : 5 files resized.
dog.0.jpg
dog.1.jpg
dog.10.jpg
dog.100.jpg
dog.101.jpg
dogs : 5 files resized.


In [73]:
testset.dic

{}